# Chapter 6: ハミルトニアンモンテカルロ法（HMC）完全ガイド

## 学習目標
- ハミルトニアンモンテカルロ法の物理学的背景を理解する
- HMCの数学的原理とアルゴリズムを習得する
- リープフロッグ積分とエネルギー保存の重要性を学ぶ
- 従来のMCMC手法との性能比較を行う
- パラメータチューニングとデバッグ手法を身につける
- 実際の問題への応用例を通じて実践的スキルを習得する

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize
from scipy.linalg import cholesky, solve_triangular
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")
np.random.seed(42)

print("HMC教育モジュールを開始します...")

## 7.1 物理学的直感：なぜハミルトニアンなのか？

### 従来のMCMC手法の限界

従来のランダムウォーク・メトロポリス・ヘイスティングス法は「酔っ払いの歩行」に例えられます。高次元空間では、ランダムな提案の大部分が低確率領域に向かってしまい、探索が極めて非効率になります。

In [ ]:
def visualize_random_walk_problems():
    """従来のランダムウォークMHの問題点を可視化"""
    
    def random_walk_trajectory(steps=1000):
        """酔っ払いの歩行をシミュレート"""
        position = np.array([0.0, 0.0])
        trajectory = [position.copy()]
        
        for _ in range(steps):
            # ランダムな方向に小さなステップ
            step = np.random.normal(0, 0.5, 2)
            position += step
            trajectory.append(position.copy())
        
        return np.array(trajectory)
    
    trajectory = random_walk_trajectory()
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # 軌跡の可視化
    axes[0].plot(trajectory[:, 0], trajectory[:, 1], 'b-', alpha=0.6, linewidth=0.5)
    axes[0].plot(trajectory[0, 0], trajectory[0, 1], 'go', markersize=8, label='開始')
    axes[0].plot(trajectory[-1, 0], trajectory[-1, 1], 'ro', markersize=8, label='終了')
    axes[0].set_title('ランダムウォークの軌跡\n（非効率な探索）')
    axes[0].set_xlabel('X1')
    axes[0].set_ylabel('X2')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 時系列
    axes[1].plot(trajectory[:200, 0], alpha=0.8, label='X1', linewidth=0.8)
    axes[1].plot(trajectory[:200, 1], alpha=0.8, label='X2', linewidth=0.8)
    axes[1].set_title('座標の時系列\n（強い自己相関）')
    axes[1].set_xlabel('ステップ')
    axes[1].set_ylabel('値')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("ランダムウォークの問題点：")
    print("1. 小さなステップサイズ → 探索が遅い")
    print("2. ランダムな方向 → 効率的でない動き")
    print("3. 強い自己相関 → 独立サンプルが少ない")

visualize_random_walk_problems()

### ハミルトニアン力学の基本概念

HMCは古典力学のハミルトニアン形式から着想を得ています：

- **従来のMCMC**: 目隠しをした人がランダムに歩き回る
- **HMC**: 坂道を転がるボールの物理的な動き

物理学と統計学の対応関係：
- **位置 q**: パラメータ（推定したい量）
- **運動量 p**: 補助変数（サンプリング用）
- **ポテンシャルエネルギー U(q)**: 負の対数事後確率 -log π(q)
- **運動エネルギー K(p)**: 補助変数の二次形式
- **力 F = -∇U(q)**: 対数確率の勾配 ∇log π(q)

In [ ]:
def visualize_hamiltonian_concept():
    """ハミルトニアン力学の概念を可視化"""
    
    t = np.linspace(0, 4*np.pi, 1000)
    
    # 位置（パラメータ）
    q = np.cos(t)
    # 運動量（補助変数）
    p = -np.sin(t)
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # 位相空間の軌跡
    axes[0, 0].plot(q, p, 'b-', linewidth=2)
    axes[0, 0].plot(q[0], p[0], 'go', markersize=8, label='開始')
    axes[0, 0].plot(q[-1], p[-1], 'ro', markersize=8, label='終了')
    axes[0, 0].set_xlabel('位置 q（パラメータ）')
    axes[0, 0].set_ylabel('運動量 p（補助変数）')
    axes[0, 0].set_title('位相空間での軌跡\n（エネルギー保存による円軌道）')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].set_aspect('equal')
    
    # 時系列
    axes[0, 1].plot(t, q, label='位置 q', linewidth=2)
    axes[0, 1].plot(t, p, label='運動量 p', linewidth=2)
    axes[0, 1].set_xlabel('時間')
    axes[0, 1].set_ylabel('値')
    axes[0, 1].set_title('時間発展')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # エネルギーの時間変化
    energy = 0.5 * (q**2 + p**2)
    axes[1, 0].plot(t, energy, 'r-', linewidth=2)
    axes[1, 0].set_xlabel('時間')
    axes[1, 0].set_ylabel('総エネルギー H(q,p)')
    axes[1, 0].set_title('ハミルトニアン（総エネルギー）\n理想的には一定値')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 確率密度との対応
    q_range = np.linspace(-3, 3, 100)
    prob_density = np.exp(-0.5 * q_range**2)  # 正規分布
    axes[1, 1].plot(q_range, prob_density, 'k-', linewidth=2, label='目標分布')
    axes[1, 1].hist(q[::10], bins=30, density=True, alpha=0.7, 
                   color='blue', label='HMCサンプル')
    axes[1, 1].set_xlabel('位置 q')
    axes[1, 1].set_ylabel('確率密度')
    axes[1, 1].set_title('目標分布の再現')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("ハミルトニアン力学の特徴：")
    print("1. エネルギー保存：系の総エネルギーが保存される")
    print("2. 位相空間：(位置, 運動量)の組み合わせで状態を表現")
    print("3. 決定論的発展：初期条件が決まれば軌跡が一意に決まる")
    print("4. 可逆性：時間を逆向きに進めることが可能")

visualize_hamiltonian_concept()

## 7.2 HMCの数学的基礎

### ハミルトニアンの定義

**MCMCにおけるハミルトニアン**：
```
H(q, p) = U(q) + K(p)
```

- **U(q)**: ポテンシャルエネルギー = -log π(q)
- **K(p)**: 運動エネルギー = (1/2)p^T M^(-1) p

### ハミルトンの運動方程式

```
dq/dt = ∂H/∂p = M^(-1) p    （位置の時間微分）
dp/dt = -∂H/∂q = -∇U(q)     （運動量の時間微分）
```

In [ ]:
def hamilton_equations_demo():
    """ハミルトンの運動方程式のデモンストレーション"""
    
    print("ハミルトンの運動方程式：")
    print("dq/dt = ∂H/∂p = M^(-1) p    （位置の時間微分）")
    print("dp/dt = -∂H/∂q = -∇U(q)     （運動量の時間微分）")
    print()
    
    # 1次元調和振動子の例
    def harmonic_oscillator_exact(t, q0, p0):
        """解析解"""
        q = q0 * np.cos(t) + p0 * np.sin(t)
        p = -q0 * np.sin(t) + p0 * np.cos(t)
        return q, p
    
    def harmonic_oscillator_euler(dt, steps, q0, p0):
        """オイラー法による数値解"""
        q, p = q0, p0
        trajectory = [(q, p)]
        
        for _ in range(steps):
            # オイラー法（不正確）
            q_new = q + dt * p
            p_new = p - dt * q
            q, p = q_new, p_new
            trajectory.append((q, p))
        
        return np.array(trajectory)
    
    def harmonic_oscillator_leapfrog(dt, steps, q0, p0):
        """リープフロッグ法による数値解"""
        q, p = q0, p0
        trajectory = [(q, p)]
        
        for _ in range(steps):
            # リープフロッグ法（シンプレクティック）
            p_half = p - 0.5 * dt * q
            q_new = q + dt * p_half  
            p_new = p_half - 0.5 * dt * q_new
            q, p = q_new, p_new
            trajectory.append((q, p))
        
        return np.array(trajectory)
    
    # 初期条件
    q0, p0 = 1.0, 0.0
    dt = 0.1
    steps = 100
    t_points = np.arange(0, (steps+1)*dt, dt)
    
    # 解析解
    q_exact, p_exact = harmonic_oscillator_exact(t_points, q0, p0)
    
    # 数値解
    euler_traj = harmonic_oscillator_euler(dt, steps, q0, p0)
    leapfrog_traj = harmonic_oscillator_leapfrog(dt, steps, q0, p0)
    
    # 可視化
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # 位相空間での軌跡
    axes[0, 0].plot(q_exact, p_exact, 'k-', linewidth=3, label='解析解', alpha=0.8)
    axes[0, 0].plot(euler_traj[:, 0], euler_traj[:, 1], 'r--', 
                   linewidth=2, label='オイラー法', alpha=0.7)
    axes[0, 0].plot(leapfrog_traj[:, 0], leapfrog_traj[:, 1], 'b:', 
                   linewidth=2, label='リープフロッグ法', alpha=0.9)
    axes[0, 0].set_xlabel('位置 q')
    axes[0, 0].set_ylabel('運動量 p')
    axes[0, 0].set_title('位相空間での軌跡比較')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].set_aspect('equal')
    
    # エネルギー保存
    energy_exact = 0.5 * (q_exact**2 + p_exact**2)
    energy_euler = 0.5 * (euler_traj[:, 0]**2 + euler_traj[:, 1]**2)
    energy_leapfrog = 0.5 * (leapfrog_traj[:, 0]**2 + leapfrog_traj[:, 1]**2)
    
    axes[0, 1].plot(t_points, energy_exact, 'k-', linewidth=3, label='解析解')
    axes[0, 1].plot(t_points, energy_euler, 'r--', linewidth=2, label='オイラー法')
    axes[0, 1].plot(t_points, energy_leapfrog, 'b:', linewidth=2, label='リープフロッグ法')
    axes[0, 1].set_xlabel('時間')
    axes[0, 1].set_ylabel('エネルギー')
    axes[0, 1].set_title('エネルギー保存性の比較')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 時系列
    axes[1, 0].plot(t_points, q_exact, 'k-', linewidth=3, label='解析解 q')
    axes[1, 0].plot(t_points, leapfrog_traj[:, 0], 'b:', linewidth=2, label='リープフロッグ q')
    axes[1, 0].set_xlabel('時間')
    axes[1, 0].set_ylabel('位置 q')
    axes[1, 0].set_title('位置の時間発展')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # エネルギー誤差
    axes[1, 1].plot(t_points, np.abs(energy_euler - energy_exact[0]), 
                   'r--', linewidth=2, label='オイラー法')
    axes[1, 1].plot(t_points, np.abs(energy_leapfrog - energy_exact[0]), 
                   'b:', linewidth=2, label='リープフロッグ法')
    axes[1, 1].set_xlabel('時間')
    axes[1, 1].set_ylabel('|エネルギー誤差|')
    axes[1, 1].set_title('エネルギー保存誤差')
    axes[1, 1].set_yscale('log')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"最終エネルギー誤差：")
    print(f"オイラー法:     {abs(energy_euler[-1] - energy_exact[0]):.6f}")
    print(f"リープフロッグ法: {abs(energy_leapfrog[-1] - energy_exact[0]):.6f}")

hamilton_equations_demo()

## 7.3 HMCアルゴリズムの完全実装

### 基本的なHMCステップ

1. **運動量のサンプリング**: p ~ N(0, M)
2. **リープフロッグ積分**: ハミルトンの運動方程式を数値的に解く
3. **メトロポリス受理**: エネルギー差に基づいて受理/棄却を決定

In [ ]:
class ComprehensiveHMC:
    """
    教育目的の包括的HMC実装
    デバッグ情報と詳細な説明付き
    """
    
    def __init__(self, log_prob_fn, grad_log_prob_fn, mass_matrix=None):
        """
        Parameters:
        - log_prob_fn: 対数確率密度関数 log π(q)
        - grad_log_prob_fn: 勾配関数 ∇log π(q)  
        - mass_matrix: 質量行列 M（デフォルトは単位行列）
        """
        self.log_prob_fn = log_prob_fn
        self.grad_log_prob_fn = grad_log_prob_fn
        self.mass_matrix = mass_matrix
        
        # 統計情報
        self.stats = {
            'n_accepted': 0,
            'n_proposed': 0,
            'hamiltonian_errors': [],
            'step_sizes_used': [],
            'accept_probs': []
        }
    
    def sample_momentum(self, dim):
        """運動量のサンプリング"""
        if self.mass_matrix is None:
            return np.random.normal(0, 1, dim)
        else:
            return np.random.multivariate_normal(
                np.zeros(dim), self.mass_matrix
            )
    
    def kinetic_energy(self, p):
        """運動エネルギーの計算"""
        if self.mass_matrix is None:
            return 0.5 * np.sum(p**2)
        else:
            return 0.5 * p.T @ np.linalg.solve(self.mass_matrix, p)
    
    def potential_energy(self, q):
        """ポテンシャルエネルギーの計算"""
        return -self.log_prob_fn(q)
    
    def hamiltonian(self, q, p):
        """ハミルトニアンの計算"""
        return self.potential_energy(q) + self.kinetic_energy(p)
    
    def leapfrog_step(self, q, p, epsilon):
        """リープフロッグ積分による1ステップ"""
        # 運動量の半ステップ更新
        p_half = p + 0.5 * epsilon * self.grad_log_prob_fn(q)
        
        # 位置の全ステップ更新
        if self.mass_matrix is None:
            q_new = q + epsilon * p_half
        else:
            q_new = q + epsilon * np.linalg.solve(self.mass_matrix, p_half)
        
        # 運動量の残り半ステップ更新
        p_new = p_half + 0.5 * epsilon * self.grad_log_prob_fn(q_new)
        
        return q_new, p_new
    
    def leapfrog_trajectory(self, q_initial, p_initial, epsilon, L):
        """完全なリープフロッグ軌跡の計算"""
        q, p = q_initial.copy(), p_initial.copy()
        trajectory = [(q.copy(), p.copy())]
        
        for step in range(L):
            q, p = self.leapfrog_step(q, p, epsilon)
            trajectory.append((q.copy(), p.copy()))
        
        return q, p, trajectory
    
    def hmc_step(self, q_current, epsilon, L, verbose=False):
        """
        1回のHMCステップ
        
        Returns:
        - q_new: 新しい位置
        - accepted: 受理されたかどうか
        - info: 詳細情報辞書
        """
        dim = len(q_current)
        
        # 1. 運動量のサンプリング
        p_current = self.sample_momentum(dim)
        
        # 2. 現在のハミルトニアン
        H_current = self.hamiltonian(q_current, p_current)
        
        # 3. リープフロッグ積分
        q_proposed, p_proposed, trajectory = self.leapfrog_trajectory(
            q_current, p_current, epsilon, L
        )
        
        # 4. 運動量の反転（時間反転対称性）
        p_proposed = -p_proposed
        
        # 5. 提案後のハミルトニアン
        H_proposed = self.hamiltonian(q_proposed, p_proposed)
        
        # 6. ハミルトニアン差
        delta_H = H_proposed - H_current
        
        # 7. メトロポリス受理確率
        accept_prob = min(1.0, np.exp(-delta_H))
        
        # 8. 受理/棄却
        self.stats['n_proposed'] += 1
        if np.random.rand() < accept_prob:
            q_new = q_proposed
            accepted = True
            self.stats['n_accepted'] += 1
        else:
            q_new = q_current
            accepted = False
        
        # 9. 統計情報の更新
        self.stats['hamiltonian_errors'].append(abs(delta_H))
        self.stats['step_sizes_used'].append(epsilon)
        self.stats['accept_probs'].append(accept_prob)
        
        # 10. 詳細情報
        info = {
            'H_current': H_current,
            'H_proposed': H_proposed,
            'delta_H': delta_H,
            'accept_prob': accept_prob,
            'accepted': accepted,
            'trajectory': trajectory,
            'momentum_initial': p_current,
            'momentum_final': p_proposed
        }
        
        if verbose:
            print(f"HMCステップ詳細:")
            print(f"  現在位置: {q_current}")
            print(f"  提案位置: {q_proposed}")
            print(f"  ΔH = {delta_H:.6f}")
            print(f"  受理確率 = {accept_prob:.6f}")
            print(f"  結果: {'受理' if accepted else '棄却'}")
            print()
        
        return q_new, accepted, info
    
    def sample(self, initial_q, n_samples, epsilon, L, verbose=False):
        """
        HMCサンプリングの実行
        
        Parameters:
        - initial_q: 初期位置
        - n_samples: サンプル数
        - epsilon: ステップサイズ
        - L: リープフロッグステップ数
        - verbose: 詳細出力フラグ
        
        Returns:
        - samples: サンプル配列
        - detailed_info: 各ステップの詳細情報
        """
        dim = len(initial_q)
        samples = np.zeros((n_samples, dim))
        detailed_info = []
        
        q_current = initial_q.copy()
        
        for i in range(n_samples):
            q_current, accepted, info = self.hmc_step(
                q_current, epsilon, L, verbose and i < 5  # 最初の5ステップのみ詳細出力
            )
            
            samples[i] = q_current
            detailed_info.append(info)
            
            if (i + 1) % 100 == 0:
                current_accept_rate = self.stats['n_accepted'] / self.stats['n_proposed']
                print(f"Iteration {i+1}/{n_samples}, "
                      f"Acceptance rate: {current_accept_rate:.3f}, "
                      f"Mean |ΔH|: {np.mean(self.stats['hamiltonian_errors'][-100:]):.6f}")
        
        return samples, detailed_info
    
    def get_statistics(self):
        """統計情報の取得"""
        if self.stats['n_proposed'] == 0:
            return {}
        
        return {
            'acceptance_rate': self.stats['n_accepted'] / self.stats['n_proposed'],
            'mean_hamiltonian_error': np.mean(self.stats['hamiltonian_errors']),
            'std_hamiltonian_error': np.std(self.stats['hamiltonian_errors']),
            'mean_accept_prob': np.mean(self.stats['accept_probs']),
            'n_samples': self.stats['n_proposed']
        }

print("HMC実装クラスが定義されました。")

### HMC実装のテスト

In [ ]:
def test_comprehensive_hmc():
    """包括的HMC実装のテスト"""
    
    # 目標分布：2次元正規分布
    mu = np.array([1.0, -0.5])
    cov = np.array([[2.0, 1.2], [1.2, 1.5]])
    cov_inv = np.linalg.inv(cov)
    
    def log_prob(q):
        diff = q - mu
        return -0.5 * diff.T @ cov_inv @ diff
    
    def grad_log_prob(q):
        return -cov_inv @ (q - mu)
    
    # HMCサンプラーの作成
    hmc = ComprehensiveHMC(log_prob, grad_log_prob)
    
    print("=== HMC実装テスト開始 ===")
    print()
    
    # サンプリング実行
    initial_q = np.array([0.0, 0.0])
    samples, detailed_info = hmc.sample(
        initial_q=initial_q,
        n_samples=1000,
        epsilon=0.2,
        L=25,
        verbose=True  # 最初の5ステップの詳細を表示
    )
    
    # 統計情報の表示
    stats = hmc.get_statistics()
    print("\n=== 最終統計 ===")
    for key, value in stats.items():
        if isinstance(value, float):
            print(f"{key}: {value:.6f}")
        else:
            print(f"{key}: {value}")
    
    return samples, detailed_info, hmc

# テスト実行
samples, detailed_info, hmc_instance = test_comprehensive_hmc()

In [ ]:
# 結果の可視化
def visualize_hmc_results(samples, detailed_info, mu, cov):
    """HMC結果の包括的可視化"""
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # サンプル散布図
    burnin = 100
    clean_samples = samples[burnin:]
    
    axes[0, 0].scatter(clean_samples[:, 0], clean_samples[:, 1], alpha=0.6, s=10)
    axes[0, 0].scatter(mu[0], mu[1], color='red', s=100, marker='x', 
                      linewidth=3, label='真の平均')
    axes[0, 0].set_title('HMCサンプル')
    axes[0, 0].set_xlabel('q1')
    axes[0, 0].set_ylabel('q2')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].set_aspect('equal')
    
    # トレースプロット
    axes[0, 1].plot(samples[:500, 0], alpha=0.8, label='q1', linewidth=0.8)
    axes[0, 1].plot(samples[:500, 1], alpha=0.8, label='q2', linewidth=0.8)
    axes[0, 1].axvline(burnin, color='red', linestyle='--', alpha=0.7, label='Burn-in')
    axes[0, 1].set_title('トレースプロット')
    axes[0, 1].set_xlabel('Iteration')
    axes[0, 1].set_ylabel('Value')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # ハミルトニアン誤差
    errors = [info['delta_H'] for info in detailed_info]
    axes[0, 2].plot(np.abs(errors[:500]), alpha=0.8, linewidth=0.8)
    axes[0, 2].set_title('ハミルトニアン誤差 |ΔH|')
    axes[0, 2].set_xlabel('Iteration')
    axes[0, 2].set_ylabel('|ΔH|')
    axes[0, 2].set_yscale('log')
    axes[0, 2].grid(True, alpha=0.3)
    
    # 受理確率の分布
    accept_probs = [info['accept_prob'] for info in detailed_info]
    axes[1, 0].hist(accept_probs, bins=30, alpha=0.7, density=True)
    axes[1, 0].set_title('受理確率の分布')
    axes[1, 0].set_xlabel('受理確率')
    axes[1, 0].set_ylabel('密度')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 軌跡の例
    sample_trajectory = detailed_info[50]['trajectory']  # 50番目のステップの軌跡
    traj_q = np.array([traj[0] for traj in sample_trajectory])
    
    axes[1, 1].plot(traj_q[:, 0], traj_q[:, 1], 'bo-', markersize=4, 
                   linewidth=1.5, alpha=0.7)
    axes[1, 1].plot(traj_q[0, 0], traj_q[0, 1], 'go', markersize=8, label='開始')
    axes[1, 1].plot(traj_q[-1, 0], traj_q[-1, 1], 'ro', markersize=8, label='終了')
    axes[1, 1].set_title('リープフロッグ軌跡の例')
    axes[1, 1].set_xlabel('q1')
    axes[1, 1].set_ylabel('q2')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    axes[1, 1].set_aspect('equal')
    
    # 統計比較
    sample_mean = np.mean(clean_samples, axis=0)
    sample_cov = np.cov(clean_samples.T)
    
    comparison_data = [
        ['平均 q1', mu[0], sample_mean[0], abs(mu[0] - sample_mean[0])],
        ['平均 q2', mu[1], sample_mean[1], abs(mu[1] - sample_mean[1])],
        ['分散 q1', cov[0,0], sample_cov[0,0], abs(cov[0,0] - sample_cov[0,0])],
        ['分散 q2', cov[1,1], sample_cov[1,1], abs(cov[1,1] - sample_cov[1,1])],
        ['共分散', cov[0,1], sample_cov[0,1], abs(cov[0,1] - sample_cov[0,1])]
    ]
    
    axes[1, 2].axis('off')
    table_text = "統計比較\n\n"
    table_text += f"{'統計量':<8} {'真値':<8} {'推定値':<8} {'誤差':<8}\n"
    table_text += "-" * 40 + "\n"
    for row in comparison_data:
        table_text += f"{row[0]:<8} {row[1]:<8.3f} {row[2]:<8.3f} {row[3]:<8.3f}\n"
    
    axes[1, 2].text(0.1, 0.8, table_text, transform=axes[1, 2].transAxes,
                   fontsize=10, verticalalignment='top', fontfamily='monospace')
    
    plt.tight_layout()
    plt.show()

# 可視化実行
mu = np.array([1.0, -0.5])
cov = np.array([[2.0, 1.2], [1.2, 1.5]])
visualize_hmc_results(samples, detailed_info, mu, cov)

## 7.4 HMC vs 従来手法：性能比較

ランダムウォーク・メトロポリス・ヘイスティングス法とHMCの性能を定量的に比較します。

In [ ]:
def compare_hmc_vs_rwmh():
    """HMC vs ランダムウォークMHの性能比較"""
    
    # ランダムウォークMHの実装
    def random_walk_mh(log_prob_fn, initial_value, n_samples, step_size=0.5):
        """ランダムウォーク・メトロポリス・ヘイスティングス法"""
        dim = len(initial_value)
        samples = np.zeros((n_samples, dim))
        current = initial_value.copy()
        current_log_prob = log_prob_fn(current)
        n_accepted = 0
        
        proposal_cov = step_size**2 * np.eye(dim)
        
        for i in range(n_samples):
            # 提案
            proposed = np.random.multivariate_normal(current, proposal_cov)
            proposed_log_prob = log_prob_fn(proposed)
            
            # 受理確率
            log_alpha = proposed_log_prob - current_log_prob
            alpha = min(1.0, np.exp(log_alpha))
            
            # 受理/棄却
            if np.random.rand() < alpha:
                current = proposed
                current_log_prob = proposed_log_prob
                n_accepted += 1
            
            samples[i] = current
            
            if (i + 1) % 1000 == 0:
                print(f"Iteration {i+1}/{n_samples}, Acceptance rate: {n_accepted/(i+1):.3f}")
        
        acceptance_rate = n_accepted / n_samples
        return samples, acceptance_rate
    
    # 効率指標の計算
    def compute_efficiency_metrics(samples, burnin_frac=0.1):
        """効率指標の計算"""
        burnin = int(len(samples) * burnin_frac)
        clean_samples = samples[burnin:]
        
        # 各次元の自己相関時間を計算
        autocorr_times = []
        eff_sample_sizes = []
        
        for dim in range(clean_samples.shape[1]):
            data = clean_samples[:, dim]
            lags = min(200, len(data) // 4)
            
            # 自己相関の計算
            data_centered = data - np.mean(data)
            autocorr = np.correlate(data_centered, data_centered, mode='full')
            autocorr = autocorr[len(autocorr)//2:]
            autocorr = autocorr / autocorr[0]
            
            # 統合自己相関時間
            tau_int = 1.0
            for lag in range(1, min(len(autocorr), lags)):
                if autocorr[lag] > 0.01:
                    tau_int += 2 * autocorr[lag]
                else:
                    break
            
            autocorr_times.append(tau_int)
            eff_sample_sizes.append(len(data) / (2 * tau_int + 1))
        
        return autocorr_times, eff_sample_sizes
    
    # 目標分布の設定
    mu_target = np.array([0.0, 0.0])
    cov_target = np.array([[1.0, 0.8], [0.8, 1.0]])
    cov_inv = np.linalg.inv(cov_target)
    
    def log_prob(x):
        diff = x - mu_target
        return -0.5 * diff.T @ cov_inv @ diff
    
    def grad_log_prob(x):
        return -cov_inv @ (x - mu_target)
    
    # 比較実験の実行
    print("ランダムウォークMHサンプリング実行中...")
    rwmh_samples, rwmh_acceptance_rate = random_walk_mh(
        log_prob_fn=log_prob,
        initial_value=np.array([0.0, 0.0]),
        n_samples=5000,
        step_size=0.8
    )
    
    print("\nHMCサンプリング実行中...")
    hmc = ComprehensiveHMC(log_prob, grad_log_prob)
    hmc_samples, _ = hmc.sample(
        initial_q=np.array([0.0, 0.0]),
        n_samples=5000,
        epsilon=0.25,
        L=20
    )
    
    hmc_stats = hmc.get_statistics()
    
    print(f"\nランダムウォークMH受理率: {rwmh_acceptance_rate:.3f}")
    print(f"HMC受理率: {hmc_stats['acceptance_rate']:.3f}")
    
    # 効率比較
    hmc_autocorr, hmc_eff = compute_efficiency_metrics(hmc_samples)
    rwmh_autocorr, rwmh_eff = compute_efficiency_metrics(rwmh_samples)
    
    print(f"\n=== 効率比較 ===")
    print(f"{'Method':<12} {'Dim':<5} {'Acceptance':<12} {'Autocorr Time':<15} {'Eff Sample Size':<18} {'Efficiency':<12}")
    print("-" * 90)
    
    for dim in range(2):
        print(f"{'HMC':<12} {'X'+str(dim+1):<5} {hmc_stats['acceptance_rate']:<12.3f} "
              f"{hmc_autocorr[dim]:<15.2f} {hmc_eff[dim]:<18.1f} {hmc_eff[dim]/len(hmc_samples[500:]):<12.3f}")
        print(f"{'Random Walk':<12} {'X'+str(dim+1):<5} {rwmh_acceptance_rate:<12.3f} "
              f"{rwmh_autocorr[dim]:<15.2f} {rwmh_eff[dim]:<18.1f} {rwmh_eff[dim]/len(rwmh_samples[500:]):<12.3f}")
        print()
    
    print(f"平均効率比（HMC/RWMH）: {np.mean(hmc_eff)/np.mean(rwmh_eff):.2f}")
    
    return hmc_samples, rwmh_samples, hmc_eff, rwmh_eff

# 比較実験実行
hmc_samples, rwmh_samples, hmc_eff, rwmh_eff = compare_hmc_vs_rwmh()

In [ ]:
# 比較結果の可視化
def visualize_comparison(hmc_samples, rwmh_samples, hmc_eff, rwmh_eff):
    """HMC vs ランダムウォークMHの可視化比較"""
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 10))
    
    burnin = 500
    hmc_clean = hmc_samples[burnin:]
    rwmh_clean = rwmh_samples[burnin:]
    
    # トレースプロット比較
    axes[0, 0].plot(hmc_samples[:2000, 0], alpha=0.8, label='HMC', linewidth=0.8, color='blue')
    axes[0, 0].plot(rwmh_samples[:2000, 0], alpha=0.8, label='RWMH', linewidth=0.8, color='red')
    axes[0, 0].axvline(burnin, color='gray', linestyle='--', alpha=0.7)
    axes[0, 0].set_title('Trace Plot Comparison (X1)')
    axes[0, 0].set_xlabel('Iteration')
    axes[0, 0].set_ylabel('X1')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 散布図比較
    axes[0, 1].scatter(hmc_clean[::10, 0], hmc_clean[::10, 1], alpha=0.6, s=5, color='blue', label='HMC')
    axes[0, 1].scatter(rwmh_clean[::10, 0], rwmh_clean[::10, 1], alpha=0.6, s=5, color='red', label='RWMH')
    axes[0, 1].set_title('Sample Scatter Plot')
    axes[0, 1].set_xlabel('X1')
    axes[0, 1].set_ylabel('X2')
    axes[0, 1].legend()
    axes[0, 1].set_aspect('equal')
    
    # 自己相関比較
    lags = min(100, len(hmc_clean) // 10)
    
    # HMC自己相関
    hmc_data = hmc_clean[:, 0] - np.mean(hmc_clean[:, 0])
    hmc_autocorr = np.correlate(hmc_data, hmc_data, mode='full')
    hmc_autocorr = hmc_autocorr[len(hmc_autocorr)//2:lags]
    hmc_autocorr = hmc_autocorr / hmc_autocorr[0]
    
    # RWMH自己相関
    rwmh_data = rwmh_clean[:, 0] - np.mean(rwmh_clean[:, 0])
    rwmh_autocorr = np.correlate(rwmh_data, rwmh_data, mode='full')
    rwmh_autocorr = rwmh_autocorr[len(rwmh_autocorr)//2:lags]
    rwmh_autocorr = rwmh_autocorr / rwmh_autocorr[0]
    
    axes[0, 2].plot(hmc_autocorr, label='HMC', alpha=0.8, color='blue')
    axes[0, 2].plot(rwmh_autocorr, label='RWMH', alpha=0.8, color='red')
    axes[0, 2].axhline(0, color='k', linestyle='--', alpha=0.5)
    axes[0, 2].axhline(0.05, color='gray', linestyle='--', alpha=0.5)
    axes[0, 2].set_title('Autocorrelation Comparison (X1)')
    axes[0, 2].set_xlabel('Lag')
    axes[0, 2].set_ylabel('ACF')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)
    
    # 効率指標の比較
    methods = ['HMC', 'RWMH']
    x1_eff = [hmc_eff[0], rwmh_eff[0]]
    x2_eff = [hmc_eff[1], rwmh_eff[1]]
    
    x_pos = np.arange(len(methods))
    width = 0.35
    
    axes[0, 3].bar(x_pos - width/2, x1_eff, width, label='X1', alpha=0.7, color='lightblue')
    axes[0, 3].bar(x_pos + width/2, x2_eff, width, label='X2', alpha=0.7, color='lightcoral')
    axes[0, 3].set_title('Effective Sample Size')
    axes[0, 3].set_xlabel('Method')
    axes[0, 3].set_ylabel('Effective Sample Size')
    axes[0, 3].set_xticks(x_pos)
    axes[0, 3].set_xticklabels(methods)
    axes[0, 3].legend()
    axes[0, 3].grid(True, alpha=0.3)
    
    # 軌跡の比較（最初の200ステップ）
    hmc_traj = hmc_samples[:200]
    rwmh_traj = rwmh_samples[:200]
    
    axes[1, 0].plot(hmc_traj[:, 0], hmc_traj[:, 1], 'b-', alpha=0.7, linewidth=1, label='HMC')
    axes[1, 0].plot(hmc_traj[:, 0], hmc_traj[:, 1], 'b.', markersize=2, alpha=0.8)
    axes[1, 0].set_title('HMC Trajectory (first 200)')
    axes[1, 0].set_xlabel('X1')
    axes[1, 0].set_ylabel('X2')
    axes[1, 0].set_aspect('equal')
    
    axes[1, 1].plot(rwmh_traj[:, 0], rwmh_traj[:, 1], 'r-', alpha=0.7, linewidth=1, label='RWMH')
    axes[1, 1].plot(rwmh_traj[:, 0], rwmh_traj[:, 1], 'r.', markersize=2, alpha=0.8)
    axes[1, 1].set_title('RWMH Trajectory (first 200)')
    axes[1, 1].set_xlabel('X1')
    axes[1, 1].set_ylabel('X2')
    axes[1, 1].set_aspect('equal')
    
    # ステップサイズの効果
    step_distances_hmc = np.sqrt(np.sum(np.diff(hmc_samples[:1000], axis=0)**2, axis=1))
    step_distances_rwmh = np.sqrt(np.sum(np.diff(rwmh_samples[:1000], axis=0)**2, axis=1))
    
    axes[1, 2].hist(step_distances_hmc, bins=30, alpha=0.7, density=True, color='blue', label='HMC')
    axes[1, 2].hist(step_distances_rwmh, bins=30, alpha=0.7, density=True, color='red', label='RWMH')
    axes[1, 2].set_title('Step Size Distribution')
    axes[1, 2].set_xlabel('Step Distance')
    axes[1, 2].set_ylabel('Density')
    axes[1, 2].legend()
    
    # 探索効率（単位時間あたりの有効サンプル数）
    # 注：実際の計算時間は測定していないため、理論的な比較
    # HMCは1ステップあたりより多くの計算が必要だが、より効率的
    axes[1, 3].bar(['HMC', 'RWMH'], 
                   [np.mean(hmc_eff)/20, np.mean(rwmh_eff)],  # HMCは20倍の計算コストと仮定
                   alpha=0.7, color=['blue', 'red'])
    axes[1, 3].set_title('Computational Efficiency\n(ESS per unit computation)')
    axes[1, 3].set_ylabel('Relative Efficiency')
    axes[1, 3].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# 比較可視化実行
visualize_comparison(hmc_samples, rwmh_samples, hmc_eff, rwmh_eff)

## 7.5 実践的パラメータチューニング

HMCの性能は、ステップサイズ（ε）とリープフロッグステップ数（L）の設定に大きく依存します。最適なパラメータを見つけるための体系的手法を学びます。

In [ ]:
def hmc_parameter_optimization():
    """HMCパラメータの体系的最適化"""
    
    # 目標分布の設定（病的な条件数を持つ分布）
    dim = 3
    mu = np.zeros(dim)
    eigenvalues = np.array([10.0, 1.0, 0.1])  # 条件数100
    eigenvectors = np.random.randn(dim, dim)
    eigenvectors, _ = np.linalg.qr(eigenvectors)
    cov = eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T
    cov_inv = np.linalg.inv(cov)
    
    def log_prob(q):
        diff = q - mu
        return -0.5 * diff.T @ cov_inv @ diff
    
    def grad_log_prob(q):
        return -cov_inv @ (q - mu)
    
    print("目標分布の条件数:", np.linalg.cond(cov))
    print("固有値:", eigenvalues)
    print()
    
    # パラメータグリッドの設定
    epsilons = np.logspace(-2, -0.3, 6)  # 0.01 から 0.5
    L_values = [5, 10, 20, 30]
    
    # 結果保存用
    results_grid = np.zeros((len(epsilons), len(L_values), 4))  # accept_rate, autocorr_time, ess, mean_error
    
    print("パラメータ最適化実行中...")
    print(f"グリッド サイズ: {len(epsilons)} × {len(L_values)} = {len(epsilons) * len(L_values)} 組み合わせ")
    print()
    
    total_combinations = len(epsilons) * len(L_values)
    combination_count = 0
    
    for i, epsilon in enumerate(epsilons):
        for j, L in enumerate(L_values):
            combination_count += 1
            
            # HMCサンプリング
            hmc = ComprehensiveHMC(log_prob, grad_log_prob)
            samples, _ = hmc.sample(
                initial_q=np.zeros(dim),
                n_samples=1000,
                epsilon=epsilon,
                L=L
            )
            
            stats = hmc.get_statistics()
            
            # 自己相関時間の計算
            burnin = 200
            clean_samples = samples[burnin:]
            
            autocorr_times = []
            for d in range(dim):
                data = clean_samples[:, d]
                data = data - np.mean(data)
                autocorr = np.correlate(data, data, mode='full')
                autocorr = autocorr[len(autocorr)//2:]
                autocorr = autocorr / autocorr[0]
                
                tau_int = 1.0
                for lag in range(1, min(len(autocorr), 100)):
                    if autocorr[lag] > 0.01:
                        tau_int += 2 * autocorr[lag]
                    else:
                        break
                autocorr_times.append(tau_int)
            
            mean_autocorr_time = np.mean(autocorr_times)
            ess = len(clean_samples) / (2 * mean_autocorr_time + 1)
            
            # 結果の保存
            results_grid[i, j, 0] = stats['acceptance_rate']
            results_grid[i, j, 1] = mean_autocorr_time
            results_grid[i, j, 2] = ess
            results_grid[i, j, 3] = stats['mean_hamiltonian_error']
            
            if combination_count % 4 == 0:
                print(f"進捗: {combination_count}/{total_combinations} "
                      f"({100*combination_count/total_combinations:.1f}%)")
    
    # 最適解の特定
    best_i, best_j = np.unravel_index(np.argmax(results_grid[:, :, 2]), 
                                      results_grid[:, :, 2].shape)
    best_epsilon = epsilons[best_i]
    best_L = L_values[best_j]
    
    print(f"\n最適パラメータ: ε = {best_epsilon:.3f}, L = {best_L}")
    print(f"期待性能:")
    print(f"  受理率: {results_grid[best_i, best_j, 0]:.3f}")
    print(f"  ESS: {results_grid[best_i, best_j, 2]:.1f}")
    
    return results_grid, epsilons, L_values, best_epsilon, best_L

# パラメータ最適化の実行
optimization_results = hmc_parameter_optimization()

In [ ]:
# 最適化結果の可視化
def visualize_optimization_results(results_grid, epsilons, L_values):
    """パラメータ最適化結果の可視化"""
    
    fig = plt.figure(figsize=(16, 12))
    
    # ヒートマップ用のデータ準備
    epsilon_labels = [f"{eps:.3f}" for eps in epsilons]
    L_labels = [str(L) for L in L_values]
    
    # 1. 受理率
    ax1 = plt.subplot(2, 3, 1)
    im1 = ax1.imshow(results_grid[:, :, 0], aspect='auto', cmap='viridis', 
                     vmin=0, vmax=1)
    ax1.set_title('受理率')
    ax1.set_xlabel('リープフロッグステップ数 L')
    ax1.set_ylabel('ステップサイズ ε')
    ax1.set_xticks(range(len(L_values)))
    ax1.set_xticklabels(L_labels)
    ax1.set_yticks(range(len(epsilons)))
    ax1.set_yticklabels(epsilon_labels)
    plt.colorbar(im1, ax=ax1)
    
    # 2. 自己相関時間
    ax2 = plt.subplot(2, 3, 2)
    im2 = ax2.imshow(np.log10(results_grid[:, :, 1]), aspect='auto', cmap='viridis_r')
    ax2.set_title('自己相関時間 (log10)')
    ax2.set_xlabel('リープフロッグステップ数 L')
    ax2.set_ylabel('ステップサイズ ε')
    ax2.set_xticks(range(len(L_values)))
    ax2.set_xticklabels(L_labels)
    ax2.set_yticks(range(len(epsilons)))
    ax2.set_yticklabels(epsilon_labels)
    plt.colorbar(im2, ax=ax2)
    
    # 3. 有効サンプルサイズ
    ax3 = plt.subplot(2, 3, 3)
    im3 = ax3.imshow(results_grid[:, :, 2], aspect='auto', cmap='viridis')
    ax3.set_title('有効サンプルサイズ')
    ax3.set_xlabel('リープフロッグステップ数 L')
    ax3.set_ylabel('ステップサイズ ε')
    ax3.set_xticks(range(len(L_values)))
    ax3.set_xticklabels(L_labels)
    ax3.set_yticks(range(len(epsilons)))
    ax3.set_yticklabels(epsilon_labels)
    plt.colorbar(im3, ax=ax3)
    
    # 4. ハミルトニアン誤差
    ax4 = plt.subplot(2, 3, 4)
    im4 = ax4.imshow(np.log10(results_grid[:, :, 3]), aspect='auto', cmap='viridis_r')
    ax4.set_title('ハミルトニアン誤差 (log10)')
    ax4.set_xlabel('リープフロッグステップ数 L')
    ax4.set_ylabel('ステップサイズ ε')
    ax4.set_xticks(range(len(L_values)))
    ax4.set_xticklabels(L_labels)
    ax4.set_yticks(range(len(epsilons)))
    ax4.set_yticklabels(epsilon_labels)
    plt.colorbar(im4, ax=ax4)
    
    # 5. パレート最適解の可視化
    ax5 = plt.subplot(2, 3, 5)
    
    # 全組み合わせをプロット
    accept_rates_flat = results_grid[:, :, 0].flatten()
    ess_flat = results_grid[:, :, 2].flatten()
    
    scatter = ax5.scatter(accept_rates_flat, ess_flat, c=results_grid[:, :, 1].flatten(), 
                         cmap='viridis_r', alpha=0.7, s=50)
    
    # 最適解をハイライト
    best_i, best_j = np.unravel_index(np.argmax(results_grid[:, :, 2]), 
                                      results_grid[:, :, 2].shape)
    ax5.scatter(results_grid[best_i, best_j, 0], results_grid[best_i, best_j, 2], 
               color='red', s=100, marker='*', 
               label=f'最適解 (ε={epsilons[best_i]:.3f}, L={L_values[best_j]})')
    ax5.set_xlabel('受理率')
    ax5.set_ylabel('有効サンプルサイズ')
    ax5.set_title('パフォーマンス散布図')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax5, label='自己相関時間')
    
    # 6. 最適化サマリー
    ax6 = plt.subplot(2, 3, 6)
    ax6.axis('off')
    
    # 上位5つの組み合わせ
    ess_indices = np.unravel_index(np.argsort(results_grid[:, :, 2].flatten())[-5:],
                                   results_grid[:, :, 2].shape)
    
    summary_text = "最適パラメータ (ESS上位5位)\n\n"
    summary_text += f"{'順位':<4} {'ε':<8} {'L':<4} {'受理率':<8} {'τ':<8} {'ESS':<8}\n"
    summary_text += "-" * 50 + "\n"
    
    for rank, (i, j) in enumerate(zip(ess_indices[0][::-1], ess_indices[1][::-1])):
        eps = epsilons[i]
        L = L_values[j]
        accept_rate = results_grid[i, j, 0]
        autocorr_time = results_grid[i, j, 1]
        ess = results_grid[i, j, 2]
        
        summary_text += f"{rank+1:<4} {eps:<8.3f} {L:<4} {accept_rate:<8.3f} "
        summary_text += f"{autocorr_time:<8.2f} {ess:<8.1f}\n"
    
    best_epsilon = epsilons[best_i]
    best_L = L_values[best_j]
    summary_text += f"\n推奨設定:\n"
    summary_text += f"ε = {best_epsilon:.3f}\n"
    summary_text += f"L = {best_L}\n"
    summary_text += f"期待性能:\n"
    summary_text += f"  受理率: {results_grid[best_i, best_j, 0]:.3f}\n"
    summary_text += f"  ESS: {results_grid[best_i, best_j, 2]:.1f}\n"
    
    ax6.text(0.1, 0.9, summary_text, transform=ax6.transAxes,
            fontsize=9, verticalalignment='top', fontfamily='monospace')
    
    plt.tight_layout()
    plt.show()

# 可視化実行
results_grid, epsilons, L_values, best_epsilon, best_L = optimization_results
visualize_optimization_results(results_grid, epsilons, L_values)

## 7.6 階層ベイズモデルでのHMC応用

実際の統計問題への応用例として、学校の成績分析を行う階層ベイズモデルをHMCで解いてみます。

In [ ]:
def hierarchical_bayes_hmc_example():
    """階層ベイズモデルでのHMC応用例"""
    
    print("=== 階層ベイズモデル例：学校の成績分析 ===")
    print()
    
    # データ生成（８つの学校の成績データ）
    np.random.seed(42)
    
    # 真のパラメータ
    n_schools = 6
    true_mu = 5.0  # 全体平均
    true_tau = 2.0  # 学校間のばらつき
    true_theta = np.random.normal(true_mu, true_tau, n_schools)  # 各学校の真の平均
    school_sigmas = np.array([2.0, 1.5, 2.5, 1.8, 2.2, 1.6])  # 既知の学校内標準偏差
    
    # 観測データ
    school_sizes = [20, 15, 25, 18, 22, 16]
    observed_data = []
    
    for i in range(n_schools):
        school_data = np.random.normal(true_theta[i], school_sigmas[i], school_sizes[i])
        observed_data.append(school_data)
    
    # 観測統計量
    school_means = [np.mean(data) for data in observed_data]
    
    print("観測データサマリー:")
    print(f"{'学校':<4} {'サイズ':<6} {'観測平均':<10} {'真の平均':<10} {'標準偏差':<10}")
    print("-" * 50)
    for i in range(n_schools):
        print(f"{i+1:<4} {school_sizes[i]:<6} {school_means[i]:<10.3f} {true_theta[i]:<10.3f} {school_sigmas[i]:<10.3f}")
    print()
    
    # 階層ベイズモデルの定義
    def hierarchical_log_prob(params):
        """
        階層ベイズモデルの対数事後確率
        params = [mu, log_tau, theta_1, ..., theta_n]
        """
        mu = params[0]
        log_tau = params[1] 
        tau = np.exp(log_tau)  # 正の制約のためlog変換
        theta = params[2:2+n_schools]
        
        log_prob = 0.0
        
        # 事前分布
        log_prob += -0.5 * (mu - 0)**2 / 100**2  # μ ~ N(0, 100)
        log_prob += -0.5 * (log_tau - 0)**2 / 10**2  # log(τ) ~ N(0, 10)
        
        # 階層構造: θ_i | μ, τ ~ N(μ, τ)
        for i in range(n_schools):
            log_prob += -0.5 * (theta[i] - mu)**2 / tau**2
        
        # 尤度: y_ij | θ_i ~ N(θ_i, σ_i)
        for i in range(n_schools):
            n_i = school_sizes[i]
            y_bar_i = school_means[i]
            sigma_i = school_sigmas[i]
            
            # 十分統計量を使った尤度
            log_prob += -0.5 * n_i * (y_bar_i - theta[i])**2 / sigma_i**2
        
        return log_prob
    
    def hierarchical_grad_log_prob(params):
        """階層ベイズモデルの勾配"""
        mu = params[0]
        log_tau = params[1]
        tau = np.exp(log_tau)
        theta = params[2:2+n_schools]
        
        grad = np.zeros_like(params)
        
        # ∂/∂μ
        grad[0] = -mu / 100**2  # 事前分布
        for i in range(n_schools):
            grad[0] += (theta[i] - mu) / tau**2  # 階層構造
        
        # ∂/∂log_τ
        grad[1] = -log_tau / 10**2  # 事前分布
        grad[1] += n_schools  # |dτ/d(log τ)| = τ からのヤコビアン
        for i in range(n_schools):
            grad[1] += -(theta[i] - mu)**2 / tau**2  # 階層構造
        
        # ∂/∂θ_i
        for i in range(n_schools):
            grad[2+i] = -(theta[i] - mu) / tau**2  # 階層構造
            
            n_i = school_sizes[i]
            y_bar_i = school_means[i]
            sigma_i = school_sigmas[i]
            grad[2+i] += n_i * (y_bar_i - theta[i]) / sigma_i**2  # 尤度
        
        return grad
    
    # HMCサンプリング
    print("階層ベイズモデルのHMCサンプリング実行中...")
    
    # 初期値
    initial_params = np.concatenate([
        [0.0],  # mu
        [0.0],  # log_tau
        school_means  # theta（観測平均で初期化）
    ])
    
    hmc = ComprehensiveHMC(hierarchical_log_prob, hierarchical_grad_log_prob)
    samples, detailed_info = hmc.sample(
        initial_q=initial_params,
        n_samples=2000,
        epsilon=0.02,
        L=30
    )
    
    stats = hmc.get_statistics()
    print(f"HMC統計: 受理率={stats['acceptance_rate']:.3f}")
    print()
    
    return samples, (true_mu, true_tau, true_theta), (school_means, school_sizes, school_sigmas)

# 階層ベイズモデルの実行
hierarchical_samples, true_params, observed_params = hierarchical_bayes_hmc_example()

In [ ]:
# 階層ベイズモデル結果の分析と可視化
def analyze_hierarchical_results(samples, true_params, observed_params):
    """階層ベイズモデル結果の分析"""
    
    true_mu, true_tau, true_theta = true_params
    school_means, school_sizes, school_sigmas = observed_params
    n_schools = len(school_means)
    
    # 結果の分析
    burnin = 500
    clean_samples = samples[burnin:]
    
    # パラメータの抽出
    mu_samples = clean_samples[:, 0]
    log_tau_samples = clean_samples[:, 1]
    tau_samples = np.exp(log_tau_samples)
    theta_samples = clean_samples[:, 2:2+n_schools]
    
    # 事後統計
    print("事後統計サマリー:")
    print(f"{'パラメータ':<12} {'真値':<10} {'事後平均':<12} {'95%信頼区間':<20}")
    print("-" * 60)
    
    # μ
    mu_mean = np.mean(mu_samples)
    mu_ci = np.percentile(mu_samples, [2.5, 97.5])
    print(f"{'μ (全体平均)':<12} {true_mu:<10.3f} {mu_mean:<12.3f} [{mu_ci[0]:.3f}, {mu_ci[1]:.3f}]")
    
    # τ
    tau_mean = np.mean(tau_samples)
    tau_ci = np.percentile(tau_samples, [2.5, 97.5])
    print(f"{'τ (学校間SD)':<12} {true_tau:<10.3f} {tau_mean:<12.3f} [{tau_ci[0]:.3f}, {tau_ci[1]:.3f}]")
    
    # θ
    for i in range(n_schools):
        theta_mean = np.mean(theta_samples[:, i])
        theta_ci = np.percentile(theta_samples[:, i], [2.5, 97.5])
        print(f"{'θ_' + str(i+1):<12} {true_theta[i]:<10.3f} {theta_mean:<12.3f} [{theta_ci[0]:.3f}, {theta_ci[1]:.3f}]")
    
    # 可視化
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    
    # μの事後分布
    axes[0, 0].hist(mu_samples, bins=50, density=True, alpha=0.7, color='blue')
    axes[0, 0].axvline(true_mu, color='red', linestyle='--', linewidth=2, label='真値')
    axes[0, 0].axvline(mu_mean, color='green', linestyle='-', linewidth=2, label='事後平均')
    axes[0, 0].set_title('μ (全体平均) の事後分布')
    axes[0, 0].set_xlabel('μ')
    axes[0, 0].set_ylabel('密度')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # τの事後分布
    axes[0, 1].hist(tau_samples, bins=50, density=True, alpha=0.7, color='blue')
    axes[0, 1].axvline(true_tau, color='red', linestyle='--', linewidth=2, label='真値')
    axes[0, 1].axvline(tau_mean, color='green', linestyle='-', linewidth=2, label='事後平均')
    axes[0, 1].set_title('τ (学校間標準偏差) の事後分布')
    axes[0, 1].set_xlabel('τ')
    axes[0, 1].set_ylabel('密度')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # トレースプロット
    axes[0, 2].plot(mu_samples[:1000], alpha=0.8, label='μ', linewidth=0.8)
    axes[0, 2].plot(tau_samples[:1000], alpha=0.8, label='τ', linewidth=0.8)
    axes[0, 2].set_title('μ, τ のトレースプロット')
    axes[0, 2].set_xlabel('Iteration')
    axes[0, 2].set_ylabel('Value')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)
    
    # 各学校のθの事後分布（最初の6校）
    for i in range(6):
        row, col = (i // 3) + 1, i % 3
        axes[row, col].hist(theta_samples[:, i], bins=30, density=True, 
                           alpha=0.7, color='blue')
        axes[row, col].axvline(true_theta[i], color='red', linestyle='--', 
                              linewidth=2, label='真値')
        axes[row, col].axvline(school_means[i], color='orange', linestyle=':', 
                              linewidth=2, label='観測平均')
        axes[row, col].axvline(np.mean(theta_samples[:, i]), color='green', 
                              linestyle='-', linewidth=2, label='事後平均')
        axes[row, col].set_title(f'学校{i+1}: θ_{i+1} の事後分布')
        axes[row, col].set_xlabel(f'θ_{i+1}')
        axes[row, col].set_ylabel('密度')
        if i == 0:
            axes[row, col].legend()
        axes[row, col].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # shrinkage効果の分析
    print("\n=== Shrinkage効果の分析 ===")
    print(f"{'学校':<4} {'観測平均':<10} {'事後平均':<10} {'Shrinkage':<12} {'真値との距離改善':<20}")
    print("-" * 65)
    
    for i in range(n_schools):
        obs_mean = school_means[i]
        post_mean = np.mean(theta_samples[:, i])
        shrinkage = abs(post_mean - obs_mean)
        
        # 真値との距離の比較
        obs_error = abs(obs_mean - true_theta[i])
        post_error = abs(post_mean - true_theta[i])
        improvement = obs_error - post_error
        
        print(f"{i+1:<4} {obs_mean:<10.3f} {post_mean:<10.3f} {shrinkage:<12.3f} {improvement:<20.3f}")
    
    # 予測分布の計算
    print("\n=== 新しい学校の予測 ===")
    
    # 新しい学校の予測分布: N(μ, τ)
    pred_samples = np.random.normal(mu_samples, tau_samples)
    pred_mean = np.mean(pred_samples)
    pred_ci = np.percentile(pred_samples, [2.5, 97.5])
    
    print(f"新しい学校の成績予測:")
    print(f"  予測平均: {pred_mean:.3f}")
    print(f"  95%予測区間: [{pred_ci[0]:.3f}, {pred_ci[1]:.3f}]")

# 結果分析実行
analyze_hierarchical_results(hierarchical_samples, true_params, observed_params)

## 7.7 HMC実践のベストプラクティス

実際にHMCを使用する際の重要なポイントをまとめます。

In [ ]:
def hmc_best_practices_guide():
    """HMC実践のベストプラクティスガイド"""
    
    print("=== HMC実践のベストプラクティス ===")
    print()
    
    best_practices = {
        "1. パラメータ初期設定": [
            "まず標準設定(ε=0.1, L=20)から開始",
            "受理率を60-80%の範囲に調整",
            "ハミルトニアン誤差をモニタリング",
            "複数の初期値でテスト"
        ],
        
        "2. 収束診断": [
            "トレースプロットによる視覚的確認",
            "Gelman-Rubin統計量 (R̂ < 1.1)",
            "有効サンプルサイズの確認",
            "複数チェーンによる検証"
        ],
        
        "3. 数値安定性": [
            "勾配の数値的検証を実施",
            "パラメータのスケーリング正規化",
            "制約の適切な処理（変数変換）",
            "オーバーフローの防止"
        ],
        
        "4. 効率向上": [
            "前処理による分布の正規化",
            "質量行列の適応的調整",
            "再パラメータ化による相関軽減",
            "適応的ステップサイズ調整"
        ],
        
        "5. 実用的考慮事項": [
            "勾配計算コストの評価",
            "メモリ使用量の管理",
            "並列化による高速化",
            "専用ライブラリの活用"
        ]
    }
    
    for category, practices in best_practices.items():
        print(f"【{category}】")
        for practice in practices:
            print(f"  • {practice}")
        print()
    
    # よくある問題と対処法
    print("=== よくある問題と対処法 ===")
    print()
    
    common_issues = {
        "低い受理率 (<50%)": {
            "原因": ["ステップサイズが大きすぎる", "軌跡が長すぎる", "分布の勾配が急峻"],
            "対策": ["εを小さくする", "Lを減らす", "質量行列による前処理"]
        },
        
        "高い自己相関": {
            "原因": ["ステップサイズが小さすぎる", "軌跡が短すぎる", "強い相関構造"],
            "対策": ["εを大きくする", "Lを増やす", "再パラメータ化"]
        },
        
        "数値不安定性": {
            "原因": ["勾配計算誤差", "スケールの違い", "制約の不適切な処理"],
            "対策": ["勾配検証", "変数変換", "ログ変換の活用"]
        }
    }
    
    for issue, details in common_issues.items():
        print(f"【{issue}】")
        print("原因:")
        for cause in details["原因"]:
            print(f"  • {cause}")
        print("対策:")
        for solution in details["対策"]:
            print(f"  • {solution}")
        print()
    
    # 推奨ライブラリとツール
    print("=== 推奨ライブラリとツール ===")
    print()
    
    libraries = {
        "PyMC": "Pythonで最も人気。直感的なモデル記述、NUTS自動調整",
        "Stan/PyStan": "高性能な専用言語。最先端のNUTS実装",
        "TensorFlow Probability": "GPU加速、深層学習との統合",
        "Pyro": "PyTorchベース、研究向けの柔軟性",
        "ArviZ": "ベイズ統計の可視化と診断専用ライブラリ"
    }
    
    for lib, description in libraries.items():
        print(f"• **{lib}**: {description}")
    
    print()
    print("=== 学習から実用への移行指針 ===")
    print()
    print("1. **理論理解段階**: 本ノートブックのような教育実装で原理を学ぶ")
    print("2. **実験段階**: 簡単なモデルで専用ライブラリに慣れる")
    print("3. **応用段階**: 実際の問題にNUTSを適用")
    print("4. **最適化段階**: カスタム質量行列や並列化を活用")
    print("5. **プロダクション段階**: 監視・診断を自動化")

hmc_best_practices_guide()

## 本章のまとめ

### 🎯 主要な学習成果

1. **物理学的直感の獲得**
   - HMCの本質は「確率の山を転がるボール」
   - エネルギー保存により効率的な長距離移動が可能
   - 勾配情報を活用した賢いサンプリング

2. **アルゴリズムの完全理解**
   - リープフロッグ積分の重要性
   - メトロポリス受理による誤差補正
   - パラメータ調整の体系的手法

3. **実践的スキルの習得**
   - 階層ベイズモデルへの応用
   - 収束診断とデバッグ手法
   - 専用ライブラリへの移行指針

### 🚀 HMCの革新性

- **効率性**: 従来手法の10-100倍の効率改善
- **スケーラビリティ**: 高次元問題への対応
- **汎用性**: 勾配が利用可能な任意の分布
- **理論的保証**: 正確なサンプリングの数学的保証

### ⚡ 次のステップ

1. **No-U-Turn Sampler (NUTS)**: HMCの自動調整版
2. **変分推論**: 近似的だが高速なベイズ推論
3. **GPU並列化**: 大規模データへの対応
4. **ベイズ深層学習**: ニューラルネットワークとの統合

HMCは現代ベイズ統計学の中核技術として、研究から実務まで幅広く活用されています。本章で学んだ知識を基に、より高度なモデリングと推論に挑戦してください！

In [ ]:
print("🎉 Chapter 7: ハミルトニアンモンテカルロ法 完了！")
print("\n次は実際のプロジェクトでHMCを活用してみましょう。")
print("PyMCやStanなどの専用ライブラリも試してみてください。")